# Modeling with GNN-ePC-SAFT

Model combining graph neural network with ePC-SAFT


## Starting point

In [ ]:
%pip install rdkit torchmetrics ml-collections polars clu jraph jaxopt -q
%pip install -U "ray[tune]" -q
%pip install torch_geometric -q
%pip install pcsaft -q

In [ ]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

In [ ]:
%pip show pcsaft | grep Version

In [ ]:
!nvidia-smi

## PNA EPCSAFT

In [ ]:
!python tuner.py --workdir=/notebooks/GNN-ePC-SAFT/ --config=configs/default.py \
--config.num_train_steps=10000 --config.log_every_steps=100\
--config.learning_rate=0.001  --config.optimizer={'adam'} --config.batch_size=128\
--config.num_para=3
#--restoredir=None \

In [ ]:
import wandb
wandb.login()

In [ ]:
!python train.py --workdir=./ --config=configs/default.py \
--config.num_train_steps={int(500000)} --config.log_every_steps=100 --config.num_para=3 \
--config.checkpoint_every_steps=2000 --config.learning_rate=0.001 --config.patience=10 \
--config.warmup_steps=500 --config.optimizer={'adam'} --config.batch_size=128 \
--config.propagation_depth=3 --config.hidden_dim=64 --config.num_mlp_layers=2 --config.pre_layers=2 --config.post_layers=1

In [ ]:
!python evaluate.py --workdir=./ --config=configs/default.py \
--config.num_train_steps={int(500000)} --config.log_every_steps=100 --config.num_para=7 \
--config.checkpoint_every_steps=2000 --config.learning_rate=0.0001 --config.patience=1000 \
--config.warmup_steps=100 --config.optimizer={'adam'} --config.batch_size=128 \
--config.propagation_depth=4 --config.hidden_dim=64 --config.num_mlp_layers=2 --config.pre_layers=1 --config.post_layers=3